In [11]:
import pandas as pd
import re
import random
import os

i need to create a dataset to train my logistical regression on

i want the following features:

- how many times appears the aliases

- proportion of alphanumeric char

- size of the text

In [2]:
df = pd.read_parquet("data02.parquet")
df.map(lambda x: len(x))

,aliases,api.2000,api.2003,api.2006,api.2009,api.2012,api.2015,api.2018
companies,,,,,,,,
Banco Comercial Português,2,90,191,169,497,983,1219,824
Galp Energia,2,66,233,181,469,964,1104,812
EDP,3,80,245,140,538,1076,1528,872
Sonae,2,130,318,239,459,1109,1400,1026
Mota-Engil,2,3,67,26,164,384,596,445


let's use bcp, 2003 to start

In [3]:
aliases = df.iloc[0,0]
texts = [i["ExtractedText"] for i in df.iloc[0,1]]

my features

In [4]:
def count_alises(aliases, text):
    alias_count = {expression: 0 for expression in aliases}
    for alias in aliases:
        # Use re.escape to handle any special characters in the expression
        pattern = re.escape(alias.lower())
        matches = re.findall(pattern, text.lower())
        alias_count[alias] = len(matches)
    return sum(alias_count.values())

def proportion_alphanumeric(text):
    alphanumeric_chars = sum(char.isalnum() for char in text)
    proportion = alphanumeric_chars / len(text)
    return proportion

def count_dates(text):
    date_pattern = r'\b(\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\d{4}[-/]\d{1,2}[-/]\d{1,2})\b'
    # 10/11/2024', '10/10/2024', '12-25-1990', '2024-11-05', '01/10/2024'
    dates = re.findall(date_pattern, text)
    date_count = len(dates)
    return date_count

def count_CAPS(text):
    words = text.split()
    uppercase_word_count = sum(1 for word in words if word.isupper())
    return uppercase_word_count

def text_size(text):
    return len(text)

lets start creating the dataset

In [10]:
def dataset_create(aliases, texts, out_name):
    dic = {"news": [], "countALI": [], "propAN": [], "countDTS": [], "txtSZ": [], "countCAPS": [], "txt": []}
    i = 0 
    #global a
    random.shuffle(texts)
    for text in texts:
        valid = int(input(text))
        #valid = int(a[i])
        print(valid, end = ", ")
        if valid not in [0,1,-1,2]:
            print("not 0 or 1")
        elif valid == -1:
            i += 1
            break
        elif valid == 2:
            i += 1
            continue
        elif valid in [0,1]:
            dic["news"].append(valid)
            dic["txt"].append(text)
            dic["countALI"].append(count_alises(aliases, text))
            dic["propAN"].append(proportion_alphanumeric(text))
            dic["countDTS"].append(count_dates(text))
            dic["txtSZ"].append(text_size(text))
            dic["countCAPS"].append(count_CAPS(text))
            i += 1
    pd.DataFrame(dic).to_csv(out_name)
    print(f"\n{i}")

# (0,2) (1,2) (3,4) (2,6) (1,6) (1,7) (3,3) (1,3) (4,1) (0,1) (4,6) # FAZER MAIS 2000, 18, 09
line, year = 4, 6
aliases = df.iloc[line,0]
texts = [i["ExtractedText"] for i in df.iloc[line,year]]

#dataset_create(aliases, texts, "logreg_mota2015.csv")

ao fazer merge dos csv:

- criar coluna com o ano e depois separar em até 2008, depois de 2009 até 2014, depois de 2015 aka 7 em 7 anos

    - associar isto à prop de alfanumericos, por causa da publicidade

acresencar features:

- **titulo:** any alias in the first 50? char OU em q % de texto aparece o primeiro alias

- **links:** quantidade de links externos, contar "http"

- **horas:** quantidade de horas que aparecem "hh:mm"

- **fonte:** nos primeiros 50? caracteres ou nos finais, ver se diz rtp, publico, dn , etc. - vai ser variavel categorica, hot encoding

    - associar esta binaria ao tamanho do texto com multiplicacao

    - atencao q pode ter mais q rtp, publico no mesmo, pode se escolher o primeiro q aparecer

    - caso nao tenha nenhum, chamar unknokn

    - associar ao titulo, pq titulos sao diferentes consoante jornais

ideia:

e se fizer treedecision em vez de logistical regression ? CUIDADO COM OVERFITTING

In [29]:
csvs = [(x, x[-8:-4]) for x in os.listdir() if x[:7] == "logreg_"]
dfs = []
for csv, year in csvs:
    df = pd.read_csv(csv, index_col=0)
    df["year"] = year
    dfs.append(df)
final_df = pd.concat(dfs, ignore_index=True)

In [33]:
sample = {}
for y in final_df.iloc[:, -1]:
    if y not in sample:
        sample[y] = 1
    else:
        sample[y] += 1
sample

{'2015': 140, '2000': 29, '2003': 143, '2018': 49, '2009': 58, '2006': 77}

In [34]:
final_df

,news,countALI,propAN,countDTS,txtSZ,countCAPS,txt,year
0,1,23,0.814458,0,7055,93,Standard and Poor`s melhora rating da EDP e da...,2015
1,0,1,0.815120,4,7156,91,As 50 Sombras de Grey - PÚBLICO Saltar para a ...,2015
2,1,22,0.825824,0,9433,60,Autoridade da Concorrência condena EDP e Sonae...,2015
3,0,1,0.814649,9,8028,92,Aquilino Ribeiro - PÚBLICO Saltar para a naveg...,2015
4,0,1,0.818654,11,8856,93,Autoridade para as Condições de Trabalho - PÚB...,2015
...,...,...,...,...,...,...,...,...
491,1,8,0.793415,17,1883,8,"rtp.pt 17/01/06 ENI continua com 33,34% da Gal...",2006
492,0,2,0.798363,25,5986,47,RTP RTPVídeo | Tempo | Infografismo | Trânsito...,2006
493,1,4,0.799881,0,5047,37,DN-Suplementos Negócios 23 de Julho de 2001 Au...,2000
494,0,2,0.794682,0,3234,34,DN 20 de Setembro de 2001 Telecomunicações l...,2000
